# Deep Q-Network (DQN)
---
In this notebook, you will implement a DQN agent with OpenAI Gym's LunarLander-v2 environment.

### 1. Import the Necessary Packages

In [1]:
import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

EXPERIMENT_NAME   = "rainbow"
EXPERIMENT_DETAIL = "WiderVRangeMoreAtoms"

### 2. Instantiate the Environment and Agent

Initialize the environment in the code cell below.

In [2]:
env = gym.make('LunarLander-v2')
env.seed(0)
print('State shape: ', env.observation_space.shape)
print('Number of actions: ', env.action_space.n)

State shape:  (8,)
Number of actions:  4


Before running the next code cell, familiarize yourself with the code in **Step 2** and **Step 3** of this notebook, along with the code in `dqn_agent.py` and `model.py`.  Once you have an understanding of how the different files work together, 
- Define a neural network architecture in `model.py` that maps states to action values.  This file is mostly empty - it's up to you to define your own deep Q-network!
- Finish the `learn` method in the `Agent` class in `dqn_agent.py`.  The sampled batch of experience tuples is already provided for you; you need only use the local and target Q-networks to compute the loss, before taking a step towards minimizing the loss.

Once you have completed the code in `dqn_agent.py` and `model.py`, run the code cell below.  (_If you end up needing to make multiple changes and get unexpected behavior, please restart the kernel and run the cells from the beginning of the notebook!_)

You can find the solution files, along with saved model weights for a trained agent, in the `solution/` folder.  (_Note that there are many ways to solve this exercise, and the "solution" is just one way of approaching the problem, to yield a trained agent._)

In [3]:
# Hyperparameters

hyperparams = {
    'seed':                101,
    'buffer_size':         int(1e5),
    'batch_size':          32,
    'start_since':         3200,
    'gamma':               0.99,
    'target_update_every': 4,
    'tau':                 1e-3,
    'lr':                  1e-4,
    'weight_decay':        0,
    'update_every':        4,
    'priority_eps':        1e-5,
    'a':                   0.5,
    'n_multisteps':        3,
    'v_min':               -500,
    'v_max':               500,
    'clip':                None,
    'n_atoms':             201,
    'initial_sigma':       0.1,
    'linear_type':         'noisy'
}

# Training Parameters

train_params = {
    'n_episodes': 2000, 'max_t':    1000,
    'eps_start':  0.,   'eps_end':  0.,  'eps_decay': 0.,
    'beta_start': 0.4,   'beta_end': 1.0
}

In [4]:
from dqn_agent import Agent

agent = Agent(state_size=8, action_size=4, **hyperparams)

### 3. Train the Agent with DQN

Run the code cell below to train the agent from scratch.  You are welcome to amend the supplied values of the parameters in the function, to try to see if you can get better performance!

In [5]:
def dqn(n_episodes=2000, max_t=1000,
        eps_start=1.0, eps_end=0.01, eps_decay=0.995,
        beta_start=0., beta_end=1.0,
        continue_after_solved=True,
        save_name="checkpoint_dueling_solved.pth"):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start                    # initialize epsilon
    
    prioritized = hasattr(agent, 'beta')  # if using prioritized experience replay, initialize beta
    if prioritized:
        print("Priority Used")
        agent.beta = beta_start
        beta_increment = (beta_end - beta_start) / n_episodes
    else:
        print("Priority Not Used")
        
    solved = False
    epi_str_max_len = len(str(n_episodes))
    for i_episode in range(1, n_episodes+1):
        state = env.reset()
        score = 0
        for t in range(max_t):
            action = agent.act(state, eps)
            next_state, reward, done, _ = env.step(action)
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break
        else: # if not done (reached max_t)
            agent.memory.reset_multisteps()
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        if prioritized:
            agent.beta = min(beta_end, agent.beta + beta_increment)
        print('\rEpisode {:>{epi_max_len}d} | Current Score: {:>7.2f} | Average Score: {:>7.2f} | Epsilon: {:>6.4f}'\
              .format(i_episode, score, np.mean(scores_window), eps, epi_max_len=epi_str_max_len), end="")
        if prioritized:
            print(' | A: {:>6.4f} | Beta: {:>6.4f}'.format(agent.a, agent.beta), end='')
        print('          ', end='')
        if i_episode % 100 == 0:
            print('\rEpisode {:>{epi_max_len}} | Current Score: {:>7.2f} | Average Score: {:>7.2f} | Epsilon: {:>6.4f}'\
                  .format(i_episode, score, np.mean(scores_window), eps, epi_max_len=epi_str_max_len), end='')
            if prioritized:
                print(' | A: {:>6.4f} | Beta: {:>6.4f}'.format(agent.a, agent.beta), end='')
            print('          ')
        if not solved and np.mean(scores_window)>=200.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), save_name)
            solved = True
            if not continue_after_solved:
                break
    return scores

scores = dqn(**train_params,
             continue_after_solved=True,
             save_name="experiment_{}_{}_solved.pth".format(EXPERIMENT_NAME, EXPERIMENT_DETAIL))

# plot the scores
plt.rcParams['figure.facecolor'] = 'w'
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

Priority Used
Episode  100 | Current Score:  -88.34 | Average Score: -169.55 | Epsilon: 0.0000 | A: 0.5000 | Beta: 0.4300          
Episode  200 | Current Score:  -93.25 | Average Score: -191.13 | Epsilon: 0.0000 | A: 0.5000 | Beta: 0.4600          
Episode  300 | Current Score: -131.71 | Average Score: -168.92 | Epsilon: 0.0000 | A: 0.5000 | Beta: 0.4900          
Episode  400 | Current Score: -303.32 | Average Score: -170.26 | Epsilon: 0.0000 | A: 0.5000 | Beta: 0.5200          
Episode  500 | Current Score: -185.63 | Average Score: -171.80 | Epsilon: 0.0000 | A: 0.5000 | Beta: 0.5500          
Episode  600 | Current Score: -125.96 | Average Score: -254.44 | Epsilon: 0.0000 | A: 0.5000 | Beta: 0.5800          
Episode  700 | Current Score: -249.16 | Average Score: -371.60 | Epsilon: 0.0000 | A: 0.5000 | Beta: 0.6100          
Episode  800 | Current Score: -559.32 | Average Score: -393.85 | Epsilon: 0.0000 | A: 0.5000 | Beta: 0.6400          
Episode  900 | Current Score: -475.08 | Av

KeyboardInterrupt: 

In [6]:
torch.save(agent.qnetwork_local.state_dict(), 'experiment_{}_{}_final.pth'.format(EXPERIMENT_NAME, EXPERIMENT_DETAIL))

In [ ]:
agent.qnetwork_local.load_state_dict(torch.load('experiment_{}_{}_final.pth'.format(EXPERIMENT_NAME, EXPERIMENT_DETAIL)))

### 4. Watch a Smart Agent!

In the next code cell, you will load the trained weights from file to watch a smart agent!

In [7]:
agent.qnetwork_local.noise(False)

for i in range(10):
    state = env.reset()
    score = 0
    for j in range(1000):
        action = agent.act(state)
        env.render()
        state, reward, done, _ = env.step(action)
        score += reward
        if done:
            break
    print("Game {} Score: {} in {} steps".format(i, score, j + 1))

agent.qnetwork_local.noise(True)
env.close()

Game 0 Score: 275.5468165602048 in 565 steps
Game 1 Score: 255.57421615425778 in 161 steps
Game 2 Score: 270.9422778045492 in 290 steps
Game 3 Score: 286.1995573765409 in 212 steps
Game 4 Score: 300.06824144795746 in 201 steps
Game 5 Score: 291.49152708229053 in 212 steps
Game 6 Score: 261.24988226550124 in 172 steps
Game 7 Score: 251.0349462653667 in 180 steps
Game 8 Score: 271.09427324697305 in 177 steps
Game 9 Score: 299.31763208015616 in 198 steps


### 5. Explore

In this exercise, you have implemented a DQN agent and demonstrated how to use it to solve an OpenAI Gym environment.  To continue your learning, you are encouraged to complete any (or all!) of the following tasks:
- Amend the various hyperparameters and network architecture to see if you can get your agent to solve the environment faster.  Once you build intuition for the hyperparameters that work well with this environment, try solving a different OpenAI Gym task with discrete actions!
- You may like to implement some improvements such as prioritized experience replay, Double DQN, or Dueling DQN! 
- Write a blog post explaining the intuition behind the DQN algorithm and demonstrating how to use it to solve an RL environment of your choosing.  

In [ ]:
def reset_env():
    state = torch.from_numpy(env.reset()).unsqueeze(0).cuda()
    with torch.no_grad():
        p = agent.qnetwork_local(state).softmax(dim=-1)
        action = np.argmax(agent.supports.mul(p).sum(dim=-1, keepdim=False).cpu().numpy())
        env.render()

    p = p.cpu().squeeze().numpy()
    supports = agent.supports.cpu().numpy()
    plt.rcParams['figure.facecolor'] = 'w'
    fig, axes = plt.subplots(2, 2, figsize=(8, 8))
    for ax in axes.reshape(-1):
        ax.grid(True)
        ax.set_ylabel("estimated probability")
        ax.set_xlabel("supports")
    axes[0, 0].set_title("do nothing")
    axes[0, 1].set_title("left engine")
    axes[1, 0].set_title("main engine")
    axes[1, 1].set_title("right engine")
    axes[0, 0].bar(x=supports, height=p[0], width=5)
    axes[0, 1].bar(x=supports, height=p[1], width=5)
    axes[1, 0].bar(x=supports, height=p[2], width=5)
    axes[1, 1].bar(x=supports, height=p[3], width=5)
    plt.tight_layout()
    
    return action

In [ ]:
def step(action, n_steps):
    print(['nothing', 'left', 'main', 'right'][action])
    score_gained = 0
    for _ in range(n_steps):
        state, reward, done, _ = env.step(action)
        score_gained += reward
        with torch.no_grad():
            state = torch.from_numpy(state).unsqueeze(0).cuda()
            p = agent.qnetwork_local(state).softmax(dim=-1)
            action = np.argmax(agent.supports.mul(p).sum(dim=-1, keepdim=False).cpu().numpy())
            env.render()
        if done:
            print(done)
            break
    print(score_gained)

    p = p.cpu().squeeze().numpy()
    supports = agent.supports.cpu().numpy()
    plt.rcParams['figure.facecolor'] = 'w'
    fig, axes = plt.subplots(2, 2, figsize=(8, 8))
    for ax in axes.reshape(-1):
        ax.grid(True)
        ax.set_ylabel("estimated probability")
        ax.set_xlabel("supports")
    axes[0, 0].set_title("do nothing")
    axes[0, 1].set_title("left engine")
    axes[1, 0].set_title("main engine")
    axes[1, 1].set_title("right engine")
    axes[0, 0].bar(x=supports, height=p[0], width=5)
    axes[0, 1].bar(x=supports, height=p[1], width=5)
    axes[1, 0].bar(x=supports, height=p[2], width=5)
    axes[1, 1].bar(x=supports, height=p[3], width=5)
    plt.tight_layout()
    
    return action

In [ ]:
action = reset_env()

In [ ]:
action = step(action)

In [ ]:
env.close()